# LabOP labware database SiLA microservice explorer

The SiLA microservice is running in a separte docker container on port 50052 via the cotainer-network.

This Jupiter notebook show some simple examples, how to use the SiLA microservices.

The SiLA microservice offers three different services:

1. LabautomationService
1. LabwareQueryService
1. LabwareService

In [ ]:
import logging
from sila2.client import SilaClient

#sila_server_host = "localhost"
sila_server_host = "labware-databank"
sila_server_port = 50052
client = SilaClient(sila_server_host, sila_server_port, insecure=True)

## Query SiLA features of SiLA server

In [ ]:
client.SiLAService.ImplementedFeatures.get()

In [ ]:
client.SiLAService.GetFeatureDefinition(FeatureIdentifier="de.unigreifswald/labware/LabwareAutomationService/v1")

## Using the Labautomation service

In [ ]:
labauto_client = client.LabwareAutomationService
labauto_client.GetLabwareDimensions(Manufacturer="Saarstedt", ProductID="123456", Unit="mm").Dimensions

In [ ]:

labauto_client.GetGrippingHeight(Manufacturer="Greiner", ProductID="123456", Unit="mm", Lidded=False).GrippingHeight

In [ ]:
labauto_client.GetGrippingHeight(Manufacturer="Greiner", ProductID="123456", Unit="mm", Lidded=True).GrippingHeight

In [ ]:
labauto_client.GetLabwareWellVolume(Manufacturer="Greiner", ProductID="123456", Unit="mm").Volume

## Using the SiLA SPARQL query service

In [ ]:
# get all labware from Greiner

query = """

PREFIX lolw: <http://www.labop.org/labware#>

SELECT ?lw_name ?lw_num_wells 
WHERE {
    ?lw_name rdf:type lolw:Labware.
    ?lw_name lolw:hasManifacturer "Greiner".
    ?lw_name lolw:hasNumWells ?lw_num_wells.
    }
"""

In [ ]:
# actual query call

res = client.LabwareQueryService.SPARQLQuery(Query=query)
res.Result

In [ ]:
# get first 100 triples

query = "SELECT * WHERE { ?s ?p ?o } LIMIT 100"

In [ ]:
import re

query = "SELECT ?lw_name ?lw_num_wells"

# regular expression to find all variables in the query

variables = re.findall(r"\?(\w+)", query)

variables

In [ ]:
import pandas as pd

df_head = pd.DataFrame(columns=variables)

res_list = [["labware.conical-50", 1.0], ["labware.96-flat-black-black-fluotrac-600", 96.0], ["labware.conical-15", 1.0], ["labware.tube-15", 1.0], ["labware.384-v-clear-clear", 384.0], ["labware.1536-white-tc", 1536.0]]

# pandas dataframe from res_list with header df_head

res_df = pd.DataFrame(columns=variables, data=res_list)


res_df

In [ ]:
# res_df in json format

res_json = res_df.to_json(orient="records")

# res_json in string format

res_json_str = res_df.to_json(orient="records")

type(res_json_str)



